# Исследование надёжности заёмщиков

## Описание

Заказчик — кредитный отдел банка. <br>
Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок.<br> 
Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели кредитного скоринга — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

**Поля таблицы:**

    children — количество детей в семье
    days_employed — общий трудовой стаж в днях
    dob_years — возраст клиента в годах
    education — уровень образования клиента
    education_id — идентификатор уровня образования
    family_status — семейное положение
    family_status_id — идентификатор семейного положения
    gender — пол клиента
    income_type — тип занятости
    debt — имел ли задолженность по возврату кредитов
    total_income — ежемесячный доход
    purpose — цель получения кредита

## Откроем файл с данными и изучим общую информацию

In [97]:
import pandas as pd

df = pd.read_csv('data.csv')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [98]:
print('Значения children\n',df['children'].value_counts())

Значения children
  0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64


In [99]:
print(f'минимальный стаж - {df.days_employed.min()}\nмаксимальный стаж - {df.days_employed.max()}')

минимальный стаж - -18388.949900568383
максимальный стаж - 401755.40047533


In [100]:
print('значения education\n',df['education'].value_counts())

значения education
 среднее                13750
высшее                  4718
СРЕДНЕЕ                  772
Среднее                  711
неоконченное высшее      668
ВЫСШЕЕ                   274
Высшее                   268
начальное                250
Неоконченное высшее       47
НЕОКОНЧЕННОЕ ВЫСШЕЕ       29
НАЧАЛЬНОЕ                 17
Начальное                 15
ученая степень             4
Ученая степень             1
УЧЕНАЯ СТЕПЕНЬ             1
Name: education, dtype: int64


In [101]:
print('значения family_status\n',df['family_status'].value_counts())

значения family_status
 женат / замужем          12380
гражданский брак          4177
Не женат / не замужем     2813
в разводе                 1195
вдовец / вдова             960
Name: family_status, dtype: int64


In [102]:
print('значения gender\n',df['gender'].value_counts())

значения gender
 F      14236
M       7288
XNA        1
Name: gender, dtype: int64


In [103]:
print('значения income_type\n',df['income_type'].value_counts())

значения income_type
 сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
студент                1
в декрете              1
Name: income_type, dtype: int64


In [104]:
print('мин-макс total_icome\n',df['total_income'].min(),'-', df['total_income'].max())

мин-макс total_icome
 20667.26379327158 - 2265604.028722744


In [105]:
print('уникальные значения purpose\n',df['purpose'].unique())

уникальные значения purpose
 ['покупка жилья' 'приобретение автомобиля' 'дополнительное образование'
 'сыграть свадьбу' 'операции с жильем' 'образование'
 'на проведение свадьбы' 'покупка жилья для семьи' 'покупка недвижимости'
 'покупка коммерческой недвижимости' 'покупка жилой недвижимости'
 'строительство собственной недвижимости' 'недвижимость'
 'строительство недвижимости' 'на покупку подержанного автомобиля'
 'на покупку своего автомобиля' 'операции с коммерческой недвижимостью'
 'строительство жилой недвижимости' 'жилье'
 'операции со своей недвижимостью' 'автомобили' 'заняться образованием'
 'сделка с подержанным автомобилем' 'получение образования' 'автомобиль'
 'свадьба' 'получение дополнительного образования' 'покупка своего жилья'
 'операции с недвижимостью' 'получение высшего образования'
 'свой автомобиль' 'сделка с автомобилем' 'профильное образование'
 'высшее образование' 'покупка жилья для сдачи' 'на покупку автомобиля'
 'ремонт жилью' 'заняться высшим образованием']


**Вывод**

- В 2 столбцах присутствуют пропуски. 
- children- отрицательные значения, 20! детей(такое возможно, но не в таком количестве)
- days_employed- отрицательные значения и пропуски
- education- поправить регистр\возможно убрать совсем т.к. рядом есть id
- family_status- поправить регистр\возможно убрать
- gender- присутствует XNA(будем считать пропуском, заменим)
- total_income- пропуски



## Предобработка данных

### Обработка пропусков

In [106]:
# children
df['children'] = abs(df['children'])
df.loc[df['children'] == 20, 'children'] = 2
df['children'].value_counts()

0    14149
1     4865
2     2131
3      330
4       41
5        9
Name: children, dtype: int64

In [107]:
#days_employed
df.loc[df['days_employed'] > 300000, 'days_employed'] = df.loc[df['days_employed'] > 300000, 'days_employed'] / 24
df['days_employed'] = df['days_employed'].fillna(df['days_employed'].mean())
df['days_employed'] = abs(df['days_employed'])
df['days_employed'].isna().sum()

0

In [108]:
#dob_years
df.loc[df['dob_years'] == 0, 'dob_years'] = df.loc[df['dob_years'] == 0, 'days_employed'] / 365 + 20

In [109]:
#education
df['education'] = df['education'].str.lower()

In [110]:
#gender
df['gender'] = df['gender'].str.lower()

In [111]:
#total_income
df['total_income'] = df['total_income'].fillna(df['total_income'].mean())

**Вывод**

- в колонке 'children' ,были значения '-1'(заменены на 1) и 20(заменены на 2 т.к 20 детей малореально в таком количестве строк)
- в колонке 'days_employed' были отрицательные значения( если считать их положительными то похоже на стаж, заменены) и слишком большие( сам не понял, с помощью слака было определено что это часы, поделены на 24). Там где аномально большие значения встречается возраст +-30 лет, что не может быть истинно, это скорее опечатка(в слаке посоветовали не обращаать на это внимание, в реальности я бы скорее удалил эти строки, в зависимости от задачи)
- в 'dob_years' были нулевые значения(опираясь на стаж заменил на \стаж в годах + 20\) 20 взял как средний возраст выхода на работу
- 'education приведен к нижнему регистру
- в 'gender' есть не определенный пол( менять не стал т.к ни на что не влияет), привел к нижнему регистру
- 'total_income' замена пропусков на средние

### Замена типа данных

In [112]:
df['days_employed'] = df['days_employed'].astype(int)
df['dob_years'] = df['dob_years'].astype(int)
df['total_income'] = df['total_income'].astype(int)

### Обработка дубликатов

In [113]:
df = df.drop_duplicates().reset_index(drop=True)
df = df.dropna().reset_index(drop=True)
df.duplicated().sum()

0

**Вывод**

- таблица приведена в нормальный вид, пригодный для работы. 
- Было 76 строк с значениями 2.0, это скорее програмная ошибка, о таком наверное следует сообщать разработчикам(это была моя ошибка). Эти строки удалены.

### Категоризация данных

#### По цели кредита

In [114]:
def category(x):
    if x['purpose'].find('жиль') >-1 or x['purpose'].find('недвижим') >-1:
        return 'недвижимость'
    elif x['purpose'].find('автомоб') >-1:
        return 'автомобиль'
    elif x['purpose'].find('свадьб') >-1:
        return 'свадьба'
    elif x['purpose'].find('образов') >-1:
        return 'образование'
    
df['cat'] = df.apply(category,axis=1)

#### По наличию детей

In [115]:
# добавление признака наличия детей
def ch(row):
    if row['children'] == 0:
        return 'нет детей'
    if row['children'] > 0:
        return 'есть дети'
df['child_group'] = df.apply(ch, axis=1)

#### По доходу

In [116]:
total_income_labels = ['доход низкий', 'доход ниже среднего', 'доход средний', 'доход выше среднего', 'доход высокий']

df['income_group_rev'] = pd.qcut(df['total_income'],
                              q=[0, .2, .4, .6, .8, 1],
                              labels=total_income_labels)
df['income_group_rev'].value_counts()

доход средний          5413
доход низкий           4291
доход ниже среднего    4291
доход высокий          4291
доход выше среднего    3168
Name: income_group_rev, dtype: int64

## Ответы на вопросы

### Есть ли зависимость между наличием детей и возвратом кредита в срок?

In [117]:
chill = df.pivot_table(index='child_group', values='debt').round(3)
chill.columns = ['не возврат']
chill

,не возврат
child_group,
есть дети,0.092
нет детей,0.075


In [118]:
no_back_no_ch = len(df[(df['children'] == 0) & (df['debt'] == 1)]) / len(df[df['children'] == 0]) * 100 #нет детей
no_back_ch = len(df[(df['children'] > 0) & (df['debt'] == 1)]) / len(df[df['children'] > 0]) * 100

print('Процент Не возврата от клиентов без детей:','%.2f' % no_back_no_ch)
print('Процент Не возврата от клиентов с детьми:','%.2f' % no_back_ch)

Процент Не возврата от клиентов без детей: 7.54
Процент Не возврата от клиентов с детьми: 9.21


**Вывод**

Клиенты с детьми имеют больший процент не возврата или просрочек.

### **Есть ли зависимость между семейным положением и возвратом кредита в срок?**

In [119]:
family = df.pivot_table(index='family_status', values='debt', aggfunc='mean')
family.columns = ['не возврат']
family.sort_values(by='не возврат', ascending=False)

,не возврат
family_status,
Не женат / не замужем,0.097509
гражданский брак,0.093471
женат / замужем,0.075452
в разводе,0.071130
вдовец / вдова,0.065693


In [120]:
#    0-женат\замужем
#    1- гражданский брак
#    2- вдовец\вдова
#    3- в разводе
#    4- не женат\не замужем
id_0 = len(df[(df['family_status_id'] == 0) & (df['debt'] == 1)]) / len(df[df['family_status_id'] == 0]) * 100
id_1 = len(df[(df['family_status_id'] == 1) & (df['debt'] == 1)]) / len(df[df['family_status_id'] == 1]) * 100
id_2 = len(df[(df['family_status_id'] == 2) & (df['debt'] == 1)]) / len(df[df['family_status_id'] == 2]) * 100
id_3 = len(df[(df['family_status_id'] == 3) & (df['debt'] == 1)]) / len(df[df['family_status_id'] == 3]) * 100
id_4 = len(df[(df['family_status_id'] == 4) & (df['debt'] == 1)]) / len(df[df['family_status_id'] == 4]) * 100

print('Процент Не возврата от клиентов женат\замужем:','%.2f' % id_0)
print('Процент Не возврата от клиентов в гразданском браке:','%.2f' % id_1)
print('Процент Не возврата от клиентов вдовец\вдова:','%.2f' % id_2)
print('Процент Не возврата от клиентов в разводе:','%.2f' % id_3)
print('Процент Не возврата от клиентов не женат\не замужем:','%.2f' % id_4)

Процент Не возврата от клиентов женат\замужем: 7.55
Процент Не возврата от клиентов в гразданском браке: 9.35
Процент Не возврата от клиентов вдовец\вдова: 6.57
Процент Не возврата от клиентов в разводе: 7.11
Процент Не возврата от клиентов не женат\не замужем: 9.75


**Вывод**

Чаще возникают проблемы у не женатых\не замужных клиентов.

### **Есть ли зависимость между уровнем дохода и возвратом кредита в срок?**

In [121]:
money = df.pivot_table(index='income_group_rev', values='debt')
money.columns = ['не возврат']
money.sort_values(by='не возврат', ascending=False)

,не возврат
income_group_rev,
доход средний,0.087567
доход выше среднего,0.084596
доход ниже среднего,0.082731
доход низкий,0.080168
доход высокий,0.069914


In [122]:
gr_1 = df[df['total_income'] < 119618]                                   # 20667-119617
gr_2 = df[(df['total_income'] > 119617) & (df['total_income'] < 167336)] # 119617-167336
gr_3 = df[(df['total_income'] > 167336) & (df['total_income'] < 256713)] # 167336-256713
gr_4 = df[df['total_income'] > 256713]                                   # 256713 -

gr_1 = len(gr_1[gr_1['debt'] == 1]) / gr_1.shape[0] * 100
gr_2 = len(gr_2[gr_2['debt'] == 1]) / gr_2.shape[0] * 100
gr_3 = len(gr_3[gr_3['debt'] == 1]) / gr_3.shape[0] * 100
gr_4 = len(gr_4[gr_4['debt'] == 1]) / gr_4.shape[0] * 100

print('Процент Не возврата от клиентов группы 1:','%.2f' % gr_1)
print('Процент Не возврата от клиентов группы 2:','%.2f' % gr_2)
print('Процент Не возврата от клиентов группы 3:','%.2f' % gr_3)
print('Процент Не возврата от клиентов группы 4:','%.2f' % gr_4)


Процент Не возврата от клиентов группы 1: 8.06
Процент Не возврата от клиентов группы 2: 8.95
Процент Не возврата от клиентов группы 3: 8.00
Процент Не возврата от клиентов группы 4: 6.94


**Вывод**

Чаще возникают проблемы у клиентов с доходом 119617-167336

### **Как разные цели кредита влияют на его возврат в срок?**

In [123]:
purp = df.pivot_table(index='cat', values='debt')
purp.columns = ['не возврат']
purp.sort_values(by='не возврат', ascending=False)

,не возврат
cat,
автомобиль,0.093590
образование,0.092200
свадьба,0.080034
недвижимость,0.072334


In [124]:
house = len(df[(df['cat'] == 'недвижимость') & (df['debt'] == 1)]) / len(df[df['cat'] == 'недвижимость']) * 100
auto = len(df[(df['cat'] == 'автомобиль') & (df['debt'] == 1)]) / len(df[df['cat'] == 'автомобиль']) * 100
educ = len(df[(df['cat'] == 'образование') & (df['debt'] == 1)]) / len(df[df['cat'] == 'образование']) * 100
wed = len(df[(df['cat'] == 'свадьба') & (df['debt'] == 1)]) / len(df[df['cat'] == 'свадьба']) * 100

print('Процент Не возврата от клиентов группы недвижимость:','%.2f' % house,
      '|всего заявок:',len(df[df['cat'] == 'недвижимость']))

print('Процент Не возврата от клиентов группы автомобиль:','%.2f' % auto, 
      '|всего заявок',len(df[df['cat'] == 'автомобиль']))

print('Процент Не возврата от клиентов группы образование:','%.2f' % educ,
      '|всего заявок:',len(df[df['cat'] == 'образование']))

print('Процент Не возврата от клиентов группы свадьба:','%.2f' % wed, 
      '|всего заявок:', len(df[df['cat'] == 'свадьба']))

Процент Не возврата от клиентов группы недвижимость: 7.23 |всего заявок: 10811
Процент Не возврата от клиентов группы автомобиль: 9.36 |всего заявок 4306
Процент Не возврата от клиентов группы образование: 9.22 |всего заявок: 4013
Процент Не возврата от клиентов группы свадьба: 8.00 |всего заявок: 2324


**Вывод**

Проблемы с кредитами возникают чаще у клиентов целью которых является приобретение автомобиля.<br>
Самые надежные клиенты планируют покупку недвижимости.

## Общий вывод

Из проведенного анализа видно что:
- cемейное положение, 
- количество детей, 
- цель кредита, 
- уровень дохода клиента<br>

**оказывают влияние на факт погашения кредита в срок.**

In [125]:
df_1 = pd.DataFrame({'категория': ['без детей', 'с детьми', 'женат\замужем',
                                   'гражданский брак', 'вдовец\вдова','в разводе',
                                   'не женат\не замужем', 'доход до 119617', 'доход 119617-167336', 
                                   'доход 167336-256713', 'доход от 256713', 'недвижимость',
                                   'автомобиль', 'образование', 'свадьба'], 
                     
                     'процент не возврата': [no_back_no_ch, no_back_ch, id_0, 
                                             id_1, id_2,id_3,id_4, gr_1,
                                             gr_2, gr_3, gr_4, house, auto, educ, wed]})
df_1 = df_1.round(2).sort_values(by ='процент не возврата', ascending=False).reset_index(drop=True)
df_1

,категория,процент не возврата
0,не женат\не замужем,9.75
1,автомобиль,9.36
2,гражданский брак,9.35
3,образование,9.22
4,с детьми,9.21
5,доход 119617-167336,8.95
6,доход до 119617,8.06
7,доход 167336-256713,8.00
8,свадьба,8.00
9,женат\замужем,7.55


In [126]:
pd.concat([purp, money, family, chill], 
          axis=0, 
          join='outer').agg({'не возврат':lambda x: x * 100}).round(2).sort_values(by='не возврат', ascending=False)

,не возврат
Не женат / не замужем,9.75
автомобиль,9.36
гражданский брак,9.35
образование,9.22
есть дети,9.20
доход средний,8.76
доход выше среднего,8.46
доход ниже среднего,8.27
доход низкий,8.02
свадьба,8.00


Из проведенного анализа видно: 
- чаще проблемы возникают у заемщиков из категории "не женат/не замужем", а самыми надежными будут клиенты из категории "вдовец/вдова"
- самой "проблемной" целью кредита является приобретение автомобиля, у приобретателей недвижимости возникает меньше проблем с возвратом
- наличие детей влияет на возврат кредита в срок